In [2]:
import numpy as np

In [5]:
#Q1 - reduce to echelon form

def ref(rowlist):
    new_rowlist = []
    
    # row indexes left to parse
    rows_left = set(range(len(rowlist)))
    
    # col indexes
    col_label_list = set(range(len(rowlist[0])))
    
    # loop through each column
    for c in col_label_list:
        # for column c, get row indexes with a nonzero at coord [r,c]
        rows_with_nonzero = [r for r in rows_left if rowlist[r][c] != 0]
        
        # if a row exists in column c with a nonzero
        if rows_with_nonzero != []:
            # take the first row found as the pivot row
            pivot = rows_with_nonzero[0]
            
            # pivot row isn't needed to be parsed anymore
            rows_left.remove(pivot)
            
            # add pivot row to the new rowlist
            new_rowlist.append(list(rowlist[pivot]))
            
            # perform ERO to get rid of nonzero values
            for r in rows_with_nonzero[1:]:
                multiplier = rowlist[r][c] / rowlist[pivot][c]
                for i in range(0, len(rowlist[r])):
                    rowlist[r][i] -= multiplier * rowlist[pivot][i]
    
    # for any leftover rows, a full row of 0s will take their place
    # at the bottom of the matrix
    for r in rows_left:
        new_rowlist.append([0] * len(col_label_list))
        
    return new_rowlist
    
# VERY IMPORTANT TO DECLARE ARRAY AS TYPE FLOAT
# VALUES WILL BE STORED AS INTEGERS IF NO TYPE IS DECLARED
rowlist = np.array([[0, 2,  3, 4, 5],
                    [1, 10, 0, 3, 2],
                    [1, 2,  3, 4, 5],
                    [0, 5,  0, 6, 7],
                    [0, 0,  0, 9, 9]], dtype = "f")

ref(rowlist)

[[1.0, 10.0, 0.0, 3.0, 2.0],
 [0.0, 2.0, 3.0, 4.0, 5.0],
 [0.0, 0.0, 15.0, 17.0, 23.0],
 [0.0, 0.0, 0.0, 4.5, 6.0],
 [0.0, 0.0, 0.0, 0.0, -3.0]]

In [10]:
# Q2 - gaussian elimination over gf2

def gf2elim(M):
    # gets the "independent" matrix and number of duplicate rows
    elim_M, blanks = remove_duplicates(M)
    
    # gets row and column lengths
    row_len, col_len = elim_M.shape
    
    # list for reordering the matrix at the end
    row_order = []
    
    # loops through the columns
    for i in range(0, col_len):
        # selects left most column
        col = elim_M.T[i]

        # list of rows that contain "1"
        one_rows = []

        # finds rows that have a leftmost "1"
        for i in range(0, len(col)):
            
            # if col has 1, only add to one_rows if it hasn't been chosen
            # as a pivot row
            if col[i] == 1 and not i in row_order:
                one_rows.append(i)

        # pivot row shall be the first row found in the column with a 1
        pivot = one_rows.pop(0) if (len(one_rows) > 0) else -1

        # perform gf2 addition on rows that have 1
        # but aren't the pivot row
        for r in one_rows:
            if not r in row_order:
                elim_M = gf2add(elim_M, r, pivot)
                
        # add the pivot row to the row_order
        if not pivot == -1:
            row_order.append(pivot)
        
    # rearrange the matrix according to the order the algorithm
    # found the pivot rows
    new_M = [elim_M[r] for r in row_order]
    
    # adds rows of 0s for every duplicate row removed earlier
    for i in range(0, blanks):
        new_M.append(np.array([0] * col_len))
        
    return new_M
    
    
# gf2 addition of r1 with r2 in matrix M
def gf2add(Ma, r1, r2):
    Ma[r1] = (Ma[r1] + Ma[r2]) % 2
    return Ma

# remove any duplicate rows in matrix M
def remove_duplicates(M):
    to_remove = []
    
    # finds any duplicate rows and adds the index to "to_remove"
    for i in range(0, len(M)):
        for j in range(i+1, len(M)):
            if np.array_equal(M[i], M[j]) and not j in to_remove:
                # inserts later rows at index 0 to prevent index
                # errors when removing
                to_remove.insert(0, j)
                
    # remove duplicate rows in matrix M
    for j in to_remove:
        M = np.delete(M, j, 0)
    
    # returns edited matrix and number of rows removed
    return [M, len(to_remove)]

In [11]:
M = np.array([[0,0,1,1],
              [1,0,1,1],
              [1,0,0,1],
              [1,1,1,1]])
              
gf2elim(M)

[array([1, 0, 1, 1]),
 array([0, 1, 0, 0]),
 array([0, 0, 1, 1]),
 array([0, 0, 0, 1])]